In [72]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

movies_merged = pd.read_csv('merged_movies.csv') 

In [73]:
tfidf_model = TfidfVectorizer(min_df = 3, max_features = None,
                     strip_accents = 'unicode', analyzer = 'word', 
                     token_pattern = r'\w{1,}', ngram_range = (1,3),
                      stop_words = 'english')
tfidf_overview = tfv.fit_transform(movies_merged['overview'].values.astype('str'))
tfidf_overview.shape

cosine_score = linear_kernel(tfidf_overview, tfidf_overview)
cosine_score

cosine_df = pd.DataFrame(cosine_score, columns = movies_merged['title'].values.tolist())
cosine_df.index = movies_merged['title'].values.tolist()
cosine_df.head()
cosine_df.iloc[0]

Avatar                                      1.000000
Pirates of the Caribbean: At World's End    0.000000
Spectre                                     0.000000
The Dark Knight Rises                       0.022512
John Carter                                 0.000000
                                              ...   
El Mariachi                                 0.000000
Newlyweds                                   0.000000
Signed, Sealed, Delivered                   0.000000
Shanghai Calling                            0.000000
My Date with Drew                           0.000000
Name: Avatar, Length: 4798, dtype: float64

In [76]:
movie_index = pd.Series(movies_merged.index, index = movies_merged['title'])
movie_titles = movies_merged['title']

def reccomend_me(movie):
    # get the index of the movie
    index = movie_index[movie]
    # convert into a list of movie index and similarity values
    similarity = list(enumerate(cosine_score[index]))
    # sort list by the similarity value
    similarity = sorted(similarity, key = lambda x: x[1], reverse = True)
    # Get the top 10 most similar movies. Exclude the first value because it is itself.
    similarity = similarity[1:11]
    recommended_index = [i[0] for i in similarity] 
    recommended_title = [movie_titles[i] for i in recommended_index]
    recommended_score = [i[1] for i in similarity]
    return pd.DataFrame({'Recommended_Movie': recommended_title,
                         'Score': recommended_score})

reccomend_me('The Dark Knight Rises')

,Recommended_Movie,Score
0,Batman Forever,0.435777
1,The Dark Knight,0.375685
2,Batman,0.330522
3,Batman Returns,0.259449
4,Slow Burn,0.197896
5,Batman Begins,0.173461
6,JFK,0.159306
7,Batman v Superman: Dawn of Justice,0.157821
8,"Batman: The Dark Knight Returns, Part 2",0.153571
9,Batman & Robin,0.147145
